https://coronavirus.ohio.gov/wps/portal/gov/covid-19/home

https://coronavirus.ohio.gov/static/COVIDSummaryData.csv


    python3 -m venv ~/v/ve/covid 
 
    . ~/v/ve/covid/bin/activate
 
    ~/v/ve/covid/bin/activate.bat
    
    pip install jupyterlab requests
    
    jupyter lab
    
    
 

In [3]:
import requests

In [4]:
help(requests)

Help on package requests:

NAME
    requests

DESCRIPTION
    Requests HTTP Library
    ~~~~~~~~~~~~~~~~~~~~~
    
    Requests is an HTTP library, written in Python, for human beings.
    Basic GET usage:
    
       >>> import requests
       >>> r = requests.get('https://www.python.org')
       >>> r.status_code
       200
       >>> b'Python is a programming language' in r.content
       True
    
    ... or POST:
    
       >>> payload = dict(key1='value1', key2='value2')
       >>> r = requests.post('https://httpbin.org/post', data=payload)
       >>> print(r.text)
       {
         ...
         "form": {
           "key1": "value1",
           "key2": "value2"
         },
         ...
       }
    
    The other HTTP methods are supported - see `requests.api`. Full documentation
    is at <https://requests.readthedocs.io>.
    
    :copyright: (c) 2017 by Kenneth Reitz.
    :license: Apache 2.0, see LICENSE for more details.

PACKAGE CONTENTS
    __version__
    _internal_utils

In [5]:
requests.get?

Signature: requests.get(url, params=None, **kwargs)
Docstring:
Sends a GET request.

:param url: URL for the new :class:`Request` object.
:param params: (optional) Dictionary, list of tuples or bytes to send
    in the query string for the :class:`Request`.
:param \*\*kwargs: Optional arguments that ``request`` takes.
:return: :class:`Response <Response>` object
:rtype: requests.Response
File:      ~/.local/share/virtualenvs/pyfri-hT7WR7Ar/lib/python3.7/site-packages/requests/api.py
Type:      function


In [6]:
response = requests.get('https://coronavirus.ohio.gov/static/COVIDSummaryData.csv')

In [7]:
response

<Response [200]>

In [8]:
download_date = datetime.date.today()

NameError: name 'datetime' is not defined

In [ ]:
download_date

In [ ]:
with open(f'covid-{download_date}.csv', 'w') as outfile:
    outfile.write(response.text)

In [ ]:
ls -l *.csv

In [ ]:
raw_text = response.text[3:]

In [ ]:
import csv

In [ ]:
reader = csv.DictReader(raw_text.splitlines())

In [ ]:
data = list(reader)

In [ ]:
data[:5]

In [ ]:
import datetime
import copy

In [ ]:
datetime.datetime.strptime('5/2/2020', "%m/%d/%Y").date()

In [ ]:
def convert_row(row):
    result = copy.copy(row)
    for field in ['Onset Date', 'Date of Death', 'Admission Date']:
        try:
            result[field] = datetime.datetime.strptime(
                row["Onset Date"], "%m/%d/%Y").date()
        except ValueError:
            result[field] = datetime.date(1900, 1, 1) 
    for field in ['Case Count', 'Death Count', 'Hospitalized Count']:
        try:
            result[field] = int(row[field])
        except ValueError:
            result[field] = None
            
    return result

In [ ]:
convert_row(data[1])

In [ ]:
len(data)

In [ ]:
cleaned = [convert_row(row) for row in data]

In [ ]:
cleaned[:5]

In [ ]:
import sqlite3

In [ ]:
import sqlite3
conn = sqlite3.connect('covid.db')
curs = conn.cursor()


In [ ]:
TABLE_CREATION_SQL = """
CREATE TABLE cases 
  (county text not null,
   sex text,
   age_range text,
   onset_date text,
   onset_epoch_sec integer,
   date_of_death text,
   death_epoch_sec integer,
   admission_date text,
   admission_epoch_sec integer,
   case_count integer,
   death_count integer,
   hospitalized_count integer,
   download_epoch_sec integer
   )
"""

In [ ]:
curs.execute(TABLE_CREATION_SQL)

In [ ]:
dt = cleaned[0]['Onset Date']

In [ ]:
int(dt)

In [ ]:
datetime.datetime.fromisoformat(str(dt)).timestamp()

In [ ]:
download_date

In [ ]:
def prepared_row(row):
    return [
        row['County'],
        row['Sex'],
        row['Age Range'],
        str(row['Onset Date']),
        datetime.datetime.fromisoformat(str(row['Onset Date'])).timestamp(),
        row['Date of Death'],
        datetime.datetime.fromisoformat(str(row['Date of Death'])).timestamp(),
        row['Admission Date'],
        datetime.datetime.fromisoformat(str(row['Admission Date'])).timestamp(),
        row['Case Count'],
        row['Death Count'],
        row['Hospitalized Count'],
        datetime.datetime.fromisoformat(str(download_date)).timestamp(),
    ]


In [ ]:
rows = [
    prepared_row(row)
    for row in cleaned
]

In [ ]:
curs.executemany('INSERT INTO cases VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)', rows)

In [ ]:
curs.commit()

In [ ]:
!ls -l *.db

In [ ]:
curs.execute('select count(*) from cases')

In [ ]:
curs.fetchall()

In [ ]:
curs.execute('select * from cases limit 10')

In [ ]:
curs.fetchall()

In [ ]:
!pip install ipython_sql

In [ ]:
%load_ext sql

In [ ]:
%sql sqlite:///covid.db

In [ ]:
%sql select * from cases limit 10